# Maximum likelihood estimation via GMM

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import formulaic as fm

from torchonometrics.gmm import GMMEstimator

np.random.seed(94305)

## OLS

In [2]:
# simulate some data for for the linear probability model
n = 1000
p = 2
X = np.random.normal(size=(n, p))
X = sm.add_constant(X)
beta = np.array([0.5, -0.5, 0.5])
y = np.random.binomial(1, 1 / (1 + np.exp(-X @ beta)))

In [3]:
lpm_mod = sm.OLS(y, X)
lpm_res = lpm_mod.fit()
print("Parameters: ", lpm_res.params)

Parameters:  [ 0.60326474 -0.11124768  0.11141277]


In [4]:
Z = X.copy()

ψ = lambda z, y, x, beta: z * (y - x @ beta)[:, np.newaxis]
gmm = GMMEstimator(ψ, weighting_matrix=None, backend = "scipy")
gmm.fit(Z, y, X)
gmm.summary()

,coef,std err,t,p-value,[0.025,0.975]
0,0.6033,0.0147,41.0456,0.0,0.5745,0.6321
1,-0.1112,0.0143,-7.8063,0.0,-0.1392,-0.0833
2,0.1114,0.0140,7.9550,0.0,0.0840,0.1389


## Logit

In [5]:
logit_mod = sm.Logit(y, X)
logit_res = logit_mod.fit(disp=0)
print("Parameters: ", logit_res.params)

Parameters:  [ 0.47180703 -0.53023967  0.5246359 ]


Moment condition:

$$
\mathbb{E} \left[ \left( y_i - \frac{1}{1 + \exp(-x_i' \beta)} \right) x_i \right] = 0
$$

In [6]:
from scipy.special import expit
ψ_logit = lambda z, y, x, beta: z * (y - expit(x @ beta))[:, np.newaxis]

gmm = GMMEstimator(ψ_logit)
gmm.fit(Z, y, X)
gmm.summary()


,coef,std err,t,p-value,[0.025,0.975]
0,0.4718,0.0147,32.1214,0.0,0.4430,0.5006
1,-0.5302,0.0142,-37.2795,0.0,-0.5581,-0.5024
2,0.5246,0.0140,37.5436,0.0,0.4973,0.5520


Identical

## Probit

In [7]:
from scipy.stats import norm
# simulate some data for probit
n, p = 1000, 2
X = np.random.normal(size=(n, p))
X = sm.add_constant(X)
beta = np.array([0.5, -0.5, 0.5])
y = np.random.binomial(1, norm.cdf(X @ beta))
Z = X.copy()

In [8]:
probit_mod = sm.Probit(y, X)
probit_res = probit_mod.fit(disp=0)
print("Parameters: ", probit_res.params)

Parameters:  [ 0.40832656 -0.4338381   0.57521853]


Moment condition
$$
\mathbb{E} \left[ \left( y_i - \Phi(x_i' \beta) \right) x_i \right] = 0
$$

In [9]:
ψ_probit = lambda z, y, x, beta: z * (y - norm.cdf(x @ beta))[:, np.newaxis]
gmm = GMMEstimator(ψ_probit)
gmm.fit(Z, y, X)
gmm.summary()

,coef,std err,t,p-value,[0.025,0.975]
0,0.4053,0.0135,29.9138,0.0,0.3788,0.4319
1,-0.4350,0.0125,-34.6831,0.0,-0.4595,-0.4104
2,0.5740,0.0125,45.8183,0.0,0.5494,0.5985


## Poisson

In [10]:
# poisson dgp
n, p = 1000, 2
X = np.random.normal(size=(n, p))
X = sm.add_constant(X)
beta = np.array([0.5, -0.5, 0.5])
y = np.random.poisson(np.exp(X @ beta))

Z = X.copy()

In [11]:
poisson_mod = sm.GLM(y, X, family=sm.families.Poisson())
poisson_res = poisson_mod.fit()
print("Parameters: ", poisson_res.params)

Parameters:  [ 0.51970481 -0.45115562  0.48937347]


In [12]:
ψ_poisson = lambda z, y, x, beta: z * (y - np.exp(x @ beta))[:, np.newaxis]
m = GMMEstimator(ψ_poisson, weighting_matrix=None, backend = "scipy")
m.fit(Z, y, X)
m.summary()

,coef,std err,t,p-value,[0.025,0.975]
0,0.5197,0.0465,11.1759,0.0,0.4286,0.6108
1,-0.4512,0.0521,-8.6614,0.0,-0.5532,-0.3491
2,0.4894,0.0562,8.7139,0.0,0.3793,0.5994
